# Author

Ronen Huang

# Import Libraries

In [1]:
import os
from sentence_transformers import SentenceTransformer
import orjson
from tqdm import tqdm
import numpy as np
from bible_search_engine.components.ranker import BiEncoderRanker
import pandas as pd

# Obtain Top 50 Bi-Encoder Query Results (Skip If Provided Relevance Scores)

In [2]:
encoded_chapters = None
chapter_ids = None
if not os.path.isfile('bible_search_engine/bible_data/encoded_chapters_niv.npy'):
    bi_encoder_model = SentenceTransformer('msmarco-distilbert-dot-v5')
    chapter_ids = []
    chapter_texts = []
    with open('bible_search_engine/bible_data/old_testament.jsonl', 'rb') as old_testament_file:
        for bible_chapter_line in tqdm(old_testament_file):
            bible_chapter = orjson.loads(bible_chapter_line)
            chapter_ids.append(bible_chapter['chapterid'])
            chapter_texts.append(' '.join(bible_chapter['verses'].values()))
    with open('bible_search_engine/bible_data/new_testament.jsonl', 'rb') as new_testament_file:
        for bible_chapter_line in tqdm(new_testament_file):
            bible_chapter = orjson.loads(bible_chapter_line)
            chapter_ids.append(bible_chapter['chapterid'])
            chapter_texts.append(' '.join(bible_chapter['verses'].values()))
    encoded_chapters = bi_encoder_model.encode(chapter_texts, show_progress_bar=True)
    np.save('bible_search_engine/bible_data/encoded_chapters.npy', encoded_chapters)
    with open('bible_search_engine/bible_data/chapterids.json', 'xb') as chapterids_file:
        chapterids_file.write(orjson.dumps(chapter_ids))
else:
    encoded_chapters = np.load('bible_search_engine/bible_data/encoded_chapters.npy')
    with open('bible_search_engine/bible_data/chapterids.json', 'rb') as chapterids_file:
        chapter_ids = orjson.loads(chapterids_file.readline())

In [3]:
chapters_dict = {}
with open('bible_data/old_testament.jsonl', 'rb') as old_testament_file:
    for bible_chapter_line in tqdm(old_testament_file):
        bible_chapter = orjson.loads(bible_chapter_line)
        chapters_dict[bible_chapter['chapterid']] = bible_chapter['chapter']
with open('bible_data/new_testament.jsonl', 'rb') as new_testament_file:
    for bible_chapter_line in tqdm(new_testament_file):
        bible_chapter = orjson.loads(bible_chapter_line)
        chapters_dict[bible_chapter['chapterid']] = bible_chapter['chapter']

929it [00:00, 43696.55it/s]
260it [00:00, 32349.06it/s]


In [4]:
bi_encoder_ranker = BiEncoderRanker('msmarco-distilbert-dot-v5', encoded_chapters, chapter_ids)

In [5]:
def query_result(ranker, query: str, chapters_dict: dict[int, str]) -> pd.DataFrame:
    top50_results = ranker.query(query)[:50]
    result_chapter_ids = []
    result_chapters = []
    for chapterid, _ in top50_results:
        result_chapter_ids.append(chapterid)
        result_chapters.append(chapters_dict[chapterid])
    return pd.DataFrame({'query': [query] * 50, 'chapter': result_chapters, 'chapterid': result_chapter_ids, 'relevance': [np.nan] * 50})

In [47]:
query_1 = query_result(bi_encoder_ranker, 'What is the significance of the covenant God made with Abraham?', chapters_dict)
if not os.path.isdir('bible_search_engine/bible_queries_relevances'):
    os.mkdir('bible_search_engine/bible_queries_relevances')
query_1.to_csv('bible_search_engine/bible_queries_relevances/query1.csv', index=None)
query_1

,query,chapter,chapterid,relevance
0,What is the significance of the covenant God m...,Deuteronomy 29,182,NaN
1,What is the significance of the covenant God m...,Hebrews 9,1142,NaN
2,What is the significance of the covenant God m...,Deuteronomy 5,158,NaN
3,What is the significance of the covenant God m...,1 Chronicles 16,354,NaN
4,What is the significance of the covenant God m...,Psalms 89,567,NaN
5,What is the significance of the covenant God m...,2 Chronicles 5,372,NaN
6,What is the significance of the covenant God m...,Psalms 105,583,NaN
7,What is the significance of the covenant God m...,1 Kings 8,299,NaN
8,What is the significance of the covenant God m...,Romans 4,1050,NaN
9,What is the significance of the covenant God m...,Hosea 12,874,NaN


In [48]:
query_2 = query_result(bi_encoder_ranker, 'If God is merciful and kind, why do people still suffer in this world?', chapters_dict)
query_2.to_csv('bible_search_engine/bible_queries_relevances/query2.csv', index=None)
query_2

,query,chapter,chapterid,relevance
0,"If God is merciful and kind, why do people sti...",Matthew 5,934,NaN
1,"If God is merciful and kind, why do people sti...",Psalms 67,545,NaN
2,"If God is merciful and kind, why do people sti...",Psalms 41,519,NaN
3,"If God is merciful and kind, why do people sti...",1 Peter 4,1155,NaN
4,"If God is merciful and kind, why do people sti...",Psalms 82,560,NaN
5,"If God is merciful and kind, why do people sti...",Psalms 117,595,NaN
6,"If God is merciful and kind, why do people sti...",Romans 5,1051,NaN
7,"If God is merciful and kind, why do people sti...",Psalms 107,585,NaN
8,"If God is merciful and kind, why do people sti...",Job 36,472,NaN
9,"If God is merciful and kind, why do people sti...",Psalms 116,594,NaN


In [72]:
query_3 = query_result(bi_encoder_ranker, 'How do the ten commandments apply today?', chapters_dict)
query_3.to_csv('bible_search_engine/bible_queries_relevances/query3.csv', index=None)
query_3

,query,chapter,chapterid,relevance
0,How do the ten commandments apply today?,Deuteronomy 8,161,NaN
1,How do the ten commandments apply today?,Joel 2,878,NaN
2,How do the ten commandments apply today?,Deuteronomy 9,162,NaN
3,How do the ten commandments apply today?,Psalms 19,497,NaN
4,How do the ten commandments apply today?,Joshua 1,188,NaN
5,How do the ten commandments apply today?,Nehemiah 1,414,NaN
6,How do the ten commandments apply today?,Psalms 110,588,NaN
7,How do the ten commandments apply today?,Deuteronomy 31,184,NaN
8,How do the ten commandments apply today?,Joshua 4,191,NaN
9,How do the ten commandments apply today?,Exodus 35,85,NaN


In [55]:
query_4 = query_result(bi_encoder_ranker, 'Where does God show up in our lives if he is not bounded by this world?', chapters_dict)
query_4.to_csv('bible_search_engine/bible_queries_relevances/query4.csv', index=None)
query_4

,query,chapter,chapterid,relevance
0,Where does God show up in our lives if he is n...,Psalms 24,502,NaN
1,Where does God show up in our lives if he is n...,1 Corinthians 4,1066,NaN
2,Where does God show up in our lives if he is n...,Psalms 97,575,NaN
3,Where does God show up in our lives if he is n...,John 7,1004,NaN
4,Where does God show up in our lives if he is n...,2 Corinthians 6,1084,NaN
5,Where does God show up in our lives if he is n...,Matthew 4,933,NaN
6,Where does God show up in our lives if he is n...,Isaiah 38,717,NaN
7,Where does God show up in our lives if he is n...,2 Corinthians 4,1082,NaN
8,Where does God show up in our lives if he is n...,Amos 7,886,NaN
9,Where does God show up in our lives if he is n...,Colossians 3,1110,NaN


In [59]:
query_5 = query_result(bi_encoder_ranker, 'Who were the kings of Israel?', chapters_dict)
query_5.to_csv('bible_search_engine/bible_queries_relevances/query5.csv', index=None)
query_5

,query,chapter,chapterid,relevance
0,Who were the kings of Israel and were they fai...,1 Kings 4,295,NaN
1,Who were the kings of Israel and were they fai...,1 Chronicles 9,347,NaN
2,Who were the kings of Israel and were they fai...,2 Chronicles 12,379,NaN
3,Who were the kings of Israel and were they fai...,Exodus 1,51,NaN
4,Who were the kings of Israel and were they fai...,1 Chronicles 23,361,NaN
5,Who were the kings of Israel and were they fai...,2 Kings 3,316,NaN
6,Who were the kings of Israel and were they fai...,1 Kings 8,299,NaN
7,Who were the kings of Israel and were they fai...,2 Kings 15,328,NaN
8,Who were the kings of Israel and were they fai...,2 Chronicles 8,375,NaN
9,Who were the kings of Israel and were they fai...,2 Samuel 5,272,NaN


In [74]:
query_6 = query_result(bi_encoder_ranker, 'What are the eternal consequences of sin?', chapters_dict)
query_6.to_csv('bible_search_engine/bible_queries_relevances/query6.csv', index=None)
query_6

,query,chapter,chapterid,relevance
0,What are the eternal consequences of sin?,Psalms 32,510,NaN
1,What are the eternal consequences of sin?,Leviticus 5,95,NaN
2,What are the eternal consequences of sin?,Romans 5,1051,NaN
3,What are the eternal consequences of sin?,Lamentations 4,801,NaN
4,What are the eternal consequences of sin?,Psalms 62,540,NaN
5,What are the eternal consequences of sin?,Psalms 74,552,NaN
6,What are the eternal consequences of sin?,Hosea 4,866,NaN
7,What are the eternal consequences of sin?,Hosea 13,875,NaN
8,What are the eternal consequences of sin?,Psalms 82,560,NaN
9,What are the eternal consequences of sin?,Psalms 85,563,NaN


In [65]:
query_7 = query_result(bi_encoder_ranker, 'Why did Jesus die on the cross?', chapters_dict)
query_7.to_csv('bible_search_engine/bible_queries_relevances/query7.csv', index=None)
query_7

,query,chapter,chapterid,relevance
0,Why did Jesus die on the cross?,John 12,1009,NaN
1,Why did Jesus die on the cross?,Romans 8,1054,NaN
2,Why did Jesus die on the cross?,Acts 12,1030,NaN
3,Why did Jesus die on the cross?,Luke 24,997,NaN
4,Why did Jesus die on the cross?,1 Corinthians 15,1077,NaN
5,Why did Jesus die on the cross?,Romans 5,1051,NaN
6,Why did Jesus die on the cross?,Hebrews 10,1143,NaN
7,Why did Jesus die on the cross?,Acts 5,1023,NaN
8,Why did Jesus die on the cross?,John 18,1015,NaN
9,Why did Jesus die on the cross?,John 11,1008,NaN


In [94]:
query_8 = query_result(bi_encoder_ranker, 'How can we be faithful to God?', chapters_dict)
query_8.to_csv('bible_search_engine/bible_queries_relevances/query8.csv', index=None)
query_8

,query,chapter,chapterid,relevance
0,How can we be faithful to God?,1 Peter 5,1156,NaN
1,How can we be faithful to God?,2 Thessalonians 1,1117,NaN
2,How can we be faithful to God?,Romans 6,1052,NaN
3,How can we be faithful to God?,2 Thessalonians 3,1119,NaN
4,How can we be faithful to God?,Titus 3,1132,NaN
5,How can we be faithful to God?,Hebrews 6,1139,NaN
6,How can we be faithful to God?,Romans 12,1058,NaN
7,How can we be faithful to God?,Psalms 89,567,NaN
8,How can we be faithful to God?,Philippians 1,1104,NaN
9,How can we be faithful to God?,2 Corinthians 8,1086,NaN


In [70]:
query_9 = query_result(bi_encoder_ranker, 'Where does God lead his people to?', chapters_dict)
query_9.to_csv('bible_search_engine/bible_queries_relevances/query9.csv', index=None)
query_9

,query,chapter,chapterid,relevance
0,Where does God lead his people to?,Psalms 24,502,NaN
1,Where does God lead his people to?,Jeremiah 23,768,NaN
2,Where does God lead his people to?,Psalms 47,525,NaN
3,Where does God lead his people to?,2 Samuel 24,291,NaN
4,Where does God lead his people to?,Psalms 99,577,NaN
5,Where does God lead his people to?,Matthew 4,933,NaN
6,Where does God lead his people to?,Psalms 23,501,NaN
7,Where does God lead his people to?,Jonah 3,892,NaN
8,Where does God lead his people to?,Psalms 11,489,NaN
9,Where does God lead his people to?,Psalms 118,596,NaN


In [88]:
query_10 = query_result(bi_encoder_ranker, 'Who are the enemies of God?', chapters_dict)
query_10.to_csv('bible_search_engine/bible_queries_relevances/query10.csv', index=None)
query_10

,query,chapter,chapterid,relevance
0,Who are the enemies of God?,James 4,1150,NaN
1,Who are the enemies of God?,Psalms 82,560,NaN
2,Who are the enemies of God?,Psalms 24,502,NaN
3,Who are the enemies of God?,Jude 1,1167,NaN
4,Who are the enemies of God?,Psalms 41,519,NaN
5,Who are the enemies of God?,Nahum 1,901,NaN
6,Who are the enemies of God?,Psalms 83,561,NaN
7,Who are the enemies of God?,Colossians 3,1110,NaN
8,Who are the enemies of God?,2 Thessalonians 1,1117,NaN
9,Who are the enemies of God?,Psalms 97,575,NaN


In [117]:
query_11 = query_result(bi_encoder_ranker, 'What did God do to unloyal servants?', chapters_dict)
query_11.to_csv('bible_search_engine/bible_queries_relevances/query11.csv', index=None)
query_11

,query,chapter,chapterid,relevance
0,What did God do to unloyal servants?,Psalms 107,585,NaN
1,What did God do to unloyal servants?,Acts 14,1032,NaN
2,What did God do to unloyal servants?,Luke 10,983,NaN
3,What did God do to unloyal servants?,Luke 19,992,NaN
4,What did God do to unloyal servants?,2 Peter 2,1158,NaN
5,What did God do to unloyal servants?,Psalms 82,560,NaN
6,What did God do to unloyal servants?,Isaiah 31,710,NaN
7,What did God do to unloyal servants?,Colossians 3,1110,NaN
8,What did God do to unloyal servants?,1 Peter 4,1155,NaN
9,What did God do to unloyal servants?,Luke 20,993,NaN


In [108]:
query_12 = query_result(bi_encoder_ranker, 'Why should we listen to authority and what are the exceptions?', chapters_dict)
query_12.to_csv('bible_search_engine/bible_queries_relevances/query12.csv', index=None)
query_12

,query,chapter,chapterid,relevance
0,Why should we listen to authority and what are...,1 Timothy 2,1121,NaN
1,Why should we listen to authority and what are...,Habakkuk 2,905,NaN
2,Why should we listen to authority and what are...,Luke 20,993,NaN
3,Why should we listen to authority and what are...,Romans 13,1059,NaN
4,Why should we listen to authority and what are...,Acts 26,1044,NaN
5,Why should we listen to authority and what are...,Matthew 15,944,NaN
6,Why should we listen to authority and what are...,Ezekiel 20,822,NaN
7,Why should we listen to authority and what are...,Psalms 130,608,NaN
8,Why should we listen to authority and what are...,Matthew 11,940,NaN
9,Why should we listen to authority and what are...,Isaiah 32,711,NaN


In [111]:
query_13 = query_result(bi_encoder_ranker, 'How do we get saved and know when we are saved?', chapters_dict)
query_13.to_csv('bible_search_engine/bible_queries_relevances/query13.csv', index=None)
query_13

,query,chapter,chapterid,relevance
0,How do we get saved and know when we are saved?,Romans 5,1051,NaN
1,How do we get saved and know when we are saved?,Romans 6,1052,NaN
2,How do we get saved and know when we are saved?,1 Peter 1,1152,NaN
3,How do we get saved and know when we are saved?,Psalms 70,548,NaN
4,How do we get saved and know when we are saved?,Psalms 20,498,NaN
5,How do we get saved and know when we are saved?,2 Corinthians 1,1079,NaN
6,How do we get saved and know when we are saved?,Acts 3,1021,NaN
7,How do we get saved and know when we are saved?,Hebrews 11,1144,NaN
8,How do we get saved and know when we are saved?,Luke 11,984,NaN
9,How do we get saved and know when we are saved?,Psalms 54,532,NaN


In [113]:
query_14 = query_result(bi_encoder_ranker, 'Where did Paul travel to spread the good news?', chapters_dict)
query_14.to_csv('bible_search_engine/bible_queries_relevances/query14.csv', index=None)
query_14

,query,chapter,chapterid,relevance
0,Where did Paul travel to spread the good news?,Acts 20,1038,NaN
1,Where did Paul travel to spread the good news?,Acts 19,1037,NaN
2,Where did Paul travel to spread the good news?,Galatians 1,1092,NaN
3,Where did Paul travel to spread the good news?,Colossians 1,1108,NaN
4,Where did Paul travel to spread the good news?,Acts 18,1036,NaN
5,Where did Paul travel to spread the good news?,Romans 1,1047,NaN
6,Where did Paul travel to spread the good news?,Ephesians 1,1098,NaN
7,Where did Paul travel to spread the good news?,Acts 21,1039,NaN
8,Where did Paul travel to spread the good news?,Acts 8,1026,NaN
9,Where did Paul travel to spread the good news?,Titus 1,1130,NaN


In [116]:
query_15 = query_result(bi_encoder_ranker, 'Who does God want us to reach out to?', chapters_dict)
query_15.to_csv('bible_search_engine/bible_queries_relevances/query15.csv', index=None)
query_15

,query,chapter,chapterid,relevance
0,Who does God want us to reach out to?,Psalms 95,573,NaN
1,Who does God want us to reach out to?,Deuteronomy 13,166,NaN
2,Who does God want us to reach out to?,Psalms 130,608,NaN
3,Who does God want us to reach out to?,Psalms 54,532,NaN
4,Who does God want us to reach out to?,1 Samuel 6,242,NaN
5,Who does God want us to reach out to?,1 Thessalonians 4,1115,NaN
6,Who does God want us to reach out to?,Luke 11,984,NaN
7,Who does God want us to reach out to?,Hebrews 6,1139,NaN
8,Who does God want us to reach out to?,Ephesians 4,1101,NaN
9,Who does God want us to reach out to?,Matthew 11,940,NaN


In [118]:
query_16 = query_result(bi_encoder_ranker, 'What does God say about spirituality?', chapters_dict)
query_16.to_csv('bible_search_engine/bible_queries_relevances/query16.csv', index=None)
query_16

,query,chapter,chapterid,relevance
0,What does God say about spirituality?,Psalms 20,498,NaN
1,What does God say about spirituality?,Romans 12,1058,NaN
2,What does God say about spirituality?,1 Corinthians 2,1064,NaN
3,What does God say about spirituality?,1 John 4,1163,NaN
4,What does God say about spirituality?,1 Timothy 4,1123,NaN
5,What does God say about spirituality?,1 Corinthians 12,1074,NaN
6,What does God say about spirituality?,Matthew 4,933,NaN
7,What does God say about spirituality?,Acts 19,1037,NaN
8,What does God say about spirituality?,Psalms 107,585,NaN
9,What does God say about spirituality?,Luke 4,977,NaN


In [96]:
query_17 = query_result(bi_encoder_ranker, 'Why did Judas betray Jesus?', chapters_dict)
query_17.to_csv('bible_search_engine/bible_queries_relevances/query17.csv', index=None)
query_17

,query,chapter,chapterid,relevance
0,Why did Judas betray Jesus?,John 18,1015,NaN
1,Why did Judas betray Jesus?,Matthew 26,955,NaN
2,Why did Judas betray Jesus?,John 13,1010,NaN
3,Why did Judas betray Jesus?,John 12,1009,NaN
4,Why did Judas betray Jesus?,Matthew 27,956,NaN
5,Why did Judas betray Jesus?,Acts 8,1026,NaN
6,Why did Judas betray Jesus?,Matthew 10,939,NaN
7,Why did Judas betray Jesus?,Matthew 15,944,NaN
8,Why did Judas betray Jesus?,Acts 12,1030,NaN
9,Why did Judas betray Jesus?,Acts 5,1023,NaN


In [97]:
query_18 = query_result(bi_encoder_ranker, 'How do we face discrimination and injustice?', chapters_dict)
query_18.to_csv('bible_search_engine/bible_queries_relevances/query18.csv', index=None)
query_18

,query,chapter,chapterid,relevance
0,How do we face discrimination and injustice?,Psalms 82,560,NaN
1,How do we face discrimination and injustice?,Lamentations 5,802,NaN
2,How do we face discrimination and injustice?,Hebrews 12,1145,NaN
3,How do we face discrimination and injustice?,Micah 2,895,NaN
4,How do we face discrimination and injustice?,1 Peter 4,1155,NaN
5,How do we face discrimination and injustice?,Zephaniah 2,908,NaN
6,How do we face discrimination and injustice?,Titus 3,1132,NaN
7,How do we face discrimination and injustice?,2 Corinthians 13,1091,NaN
8,How do we face discrimination and injustice?,Psalms 35,513,NaN
9,How do we face discrimination and injustice?,1 Peter 3,1154,NaN


In [99]:
query_19 = query_result(bi_encoder_ranker, 'Where do we find comfort and peace?', chapters_dict)
query_19.to_csv('bible_search_engine/bible_queries_relevances/query19.csv', index=None)
query_19

,query,chapter,chapterid,relevance
0,Where do we find comfort and peace?,2 Corinthians 1,1079,NaN
1,Where do we find comfort and peace?,Psalms 46,524,NaN
2,Where do we find comfort and peace?,2 Thessalonians 3,1119,NaN
3,Where do we find comfort and peace?,Psalms 122,600,NaN
4,Where do we find comfort and peace?,Matthew 5,934,NaN
5,Where do we find comfort and peace?,Psalms 128,606,NaN
6,Where do we find comfort and peace?,Isaiah 35,714,NaN
7,Where do we find comfort and peace?,Psalms 67,545,NaN
8,Where do we find comfort and peace?,Psalms 20,498,NaN
9,Where do we find comfort and peace?,Psalms 83,561,NaN


In [107]:
query_20 = query_result(bi_encoder_ranker, 'Who has received prophecies from God?', chapters_dict)
query_20.to_csv('bible_search_engine/bible_queries_relevances/query20.csv', index=None)
query_20

,query,chapter,chapterid,relevance
0,Who has received prophecies from God?,Acts 19,1037,NaN
1,Who has received prophecies from God?,Psalms 20,498,NaN
2,Who has received prophecies from God?,Luke 24,997,NaN
3,Who has received prophecies from God?,Psalms 24,502,NaN
4,Who has received prophecies from God?,Acts 5,1023,NaN
5,Who has received prophecies from God?,Mark 16,973,NaN
6,Who has received prophecies from God?,2 Peter 1,1157,NaN
7,Who has received prophecies from God?,2 Chronicles 1,368,NaN
8,Who has received prophecies from God?,Job 26,462,NaN
9,Who has received prophecies from God?,Numbers 3,120,NaN


# Create Train Query Relevance Scores

In [2]:
query_1 = pd.read_csv('bible_search_engine/bible_queries_relevances/query1.csv')
query_1

,query,chapter,chapterid,relevance
0,What is the significance of the covenant God m...,Deuteronomy 29,182,5
1,What is the significance of the covenant God m...,Hebrews 9,1142,4
2,What is the significance of the covenant God m...,Deuteronomy 5,158,2
3,What is the significance of the covenant God m...,1 Chronicles 16,354,5
4,What is the significance of the covenant God m...,Psalms 89,567,5
5,What is the significance of the covenant God m...,2 Chronicles 5,372,2
6,What is the significance of the covenant God m...,Psalms 105,583,5
7,What is the significance of the covenant God m...,1 Kings 8,299,4
8,What is the significance of the covenant God m...,Romans 4,1050,5
9,What is the significance of the covenant God m...,Hosea 12,874,1


In [3]:
query_12 = pd.read_csv('bible_search_engine/bible_queries_relevances/query12.csv')
query_12

,query,chapter,chapterid,relevance
0,Why should we listen to authority and what are...,1 Timothy 2,1121,1
1,Why should we listen to authority and what are...,Habakkuk 2,905,3
2,Why should we listen to authority and what are...,Luke 20,993,5
3,Why should we listen to authority and what are...,Romans 13,1059,5
4,Why should we listen to authority and what are...,Acts 26,1044,4
5,Why should we listen to authority and what are...,Matthew 15,944,3
6,Why should we listen to authority and what are...,Ezekiel 20,822,5
7,Why should we listen to authority and what are...,Psalms 130,608,2
8,Why should we listen to authority and what are...,Matthew 11,940,1
9,Why should we listen to authority and what are...,Isaiah 32,711,2


In [4]:
query_3 = pd.read_csv('bible_search_engine/bible_queries_relevances/query3.csv')
query_3

,query,chapter,chapterid,relevance
0,How do the ten commandments apply today?,Deuteronomy 8,161,4
1,How do the ten commandments apply today?,Joel 2,878,2
2,How do the ten commandments apply today?,Deuteronomy 9,162,5
3,How do the ten commandments apply today?,Psalms 19,497,3
4,How do the ten commandments apply today?,Joshua 1,188,5
5,How do the ten commandments apply today?,Nehemiah 1,414,4
6,How do the ten commandments apply today?,Psalms 110,588,1
7,How do the ten commandments apply today?,Deuteronomy 31,184,5
8,How do the ten commandments apply today?,Joshua 4,191,1
9,How do the ten commandments apply today?,Exodus 35,85,1


In [5]:
query_14 = pd.read_csv('bible_search_engine/bible_queries_relevances/query14.csv')
query_14

,query,chapter,chapterid,relevance
0,Where did Paul travel to spread the good news?,Acts 20,1038,5
1,Where did Paul travel to spread the good news?,Acts 19,1037,4
2,Where did Paul travel to spread the good news?,Galatians 1,1092,5
3,Where did Paul travel to spread the good news?,Colossians 1,1108,1
4,Where did Paul travel to spread the good news?,Acts 18,1036,5
5,Where did Paul travel to spread the good news?,Romans 1,1047,3
6,Where did Paul travel to spread the good news?,Ephesians 1,1098,3
7,Where did Paul travel to spread the good news?,Acts 21,1039,5
8,Where did Paul travel to spread the good news?,Acts 8,1026,1
9,Where did Paul travel to spread the good news?,Titus 1,1130,1


In [6]:
query_15 = pd.read_csv('bible_search_engine/bible_queries_relevances/query15.csv')
query_15

,query,chapter,chapterid,relevance
0,Who does God want us to reach out to?,Psalms 95,573,2
1,Who does God want us to reach out to?,Deuteronomy 13,166,1
2,Who does God want us to reach out to?,Psalms 130,608,1
3,Who does God want us to reach out to?,Psalms 54,532,1
4,Who does God want us to reach out to?,1 Samuel 6,242,1
5,Who does God want us to reach out to?,1 Thessalonians 4,1115,4
6,Who does God want us to reach out to?,Luke 11,984,3
7,Who does God want us to reach out to?,Hebrews 6,1139,1
8,Who does God want us to reach out to?,Ephesians 4,1101,5
9,Who does God want us to reach out to?,Matthew 11,940,4


In [7]:
query_6 = pd.read_csv('bible_search_engine/bible_queries_relevances/query6.csv')
query_6

,query,chapter,chapterid,relevance
0,What are the eternal consequences of sin?,Psalms 32,510,3
1,What are the eternal consequences of sin?,Leviticus 5,95,3
2,What are the eternal consequences of sin?,Romans 5,1051,5
3,What are the eternal consequences of sin?,Lamentations 4,801,4
4,What are the eternal consequences of sin?,Psalms 62,540,3
5,What are the eternal consequences of sin?,Psalms 74,552,2
6,What are the eternal consequences of sin?,Hosea 4,866,2
7,What are the eternal consequences of sin?,Hosea 13,875,4
8,What are the eternal consequences of sin?,Psalms 82,560,5
9,What are the eternal consequences of sin?,Psalms 85,563,1


In [8]:
query_17 = pd.read_csv('bible_search_engine/bible_queries_relevances/query17.csv')
query_17

,query,chapter,chapterid,relevance
0,Why did Judas betray Jesus?,John 18,1015,3
1,Why did Judas betray Jesus?,Matthew 26,955,5
2,Why did Judas betray Jesus?,John 13,1010,5
3,Why did Judas betray Jesus?,John 12,1009,5
4,Why did Judas betray Jesus?,Matthew 27,956,4
5,Why did Judas betray Jesus?,Acts 8,1026,1
6,Why did Judas betray Jesus?,Matthew 10,939,2
7,Why did Judas betray Jesus?,Matthew 15,944,1
8,Why did Judas betray Jesus?,Acts 12,1030,1
9,Why did Judas betray Jesus?,Acts 5,1023,4


In [9]:
query_8 = pd.read_csv('bible_search_engine/bible_queries_relevances/query8.csv')
query_8

,query,chapter,chapterid,relevance
0,How can we be faithful to God?,1 Peter 5,1156,5
1,How can we be faithful to God?,2 Thessalonians 1,1117,4
2,How can we be faithful to God?,Romans 6,1052,3
3,How can we be faithful to God?,2 Thessalonians 3,1119,4
4,How can we be faithful to God?,Titus 3,1132,5
5,How can we be faithful to God?,Hebrews 6,1139,2
6,How can we be faithful to God?,Romans 12,1058,4
7,How can we be faithful to God?,Psalms 89,567,3
8,How can we be faithful to God?,Philippians 1,1104,5
9,How can we be faithful to God?,2 Corinthians 8,1086,1


In [10]:
query_9 = pd.read_csv('bible_search_engine/bible_queries_relevances/query9.csv')
query_9

,query,chapter,chapterid,relevance
0,Where does God lead his people to?,Psalms 24,502,1
1,Where does God lead his people to?,Jeremiah 23,768,3
2,Where does God lead his people to?,Psalms 47,525,4
3,Where does God lead his people to?,2 Samuel 24,291,1
4,Where does God lead his people to?,Psalms 99,577,4
5,Where does God lead his people to?,Matthew 4,933,5
6,Where does God lead his people to?,Psalms 23,501,5
7,Where does God lead his people to?,Jonah 3,892,5
8,Where does God lead his people to?,Psalms 11,489,3
9,Where does God lead his people to?,Psalms 118,596,5


In [11]:
query_20 = pd.read_csv('bible_search_engine/bible_queries_relevances/query20.csv')
query_20

,query,chapter,chapterid,relevance
0,Who has received prophecies from God?,Acts 19,1037,5
1,Who has received prophecies from God?,Psalms 20,498,1
2,Who has received prophecies from God?,Luke 24,997,5
3,Who has received prophecies from God?,Psalms 24,502,1
4,Who has received prophecies from God?,Acts 5,1023,1
...,...,...,...,...
61,Who has received prophecies from God?,Jonah 1,890,5
62,Who has received prophecies from God?,Hosea 2,864,5
63,Who has received prophecies from God?,Luke 1,974,5
64,Who has received prophecies from God?,Revelation 11,1178,5


In [12]:
train_queries = pd.concat([query_1, query_12, query_3, query_14, query_15, query_6, query_17, query_8, query_9, query_20], ignore_index=True)
train_queries.to_csv('bible_search_engine/bible_queries_relevances/train_queries_relevances.csv', index=None)
train_queries

,query,chapter,chapterid,relevance
0,What is the significance of the covenant God m...,Deuteronomy 29,182,5
1,What is the significance of the covenant God m...,Hebrews 9,1142,4
2,What is the significance of the covenant God m...,Deuteronomy 5,158,2
3,What is the significance of the covenant God m...,1 Chronicles 16,354,5
4,What is the significance of the covenant God m...,Psalms 89,567,5
...,...,...,...,...
539,Who has received prophecies from God?,Jonah 1,890,5
540,Who has received prophecies from God?,Hosea 2,864,5
541,Who has received prophecies from God?,Luke 1,974,5
542,Who has received prophecies from God?,Revelation 11,1178,5


# Create Test Query Relevance Scores

In [13]:
query_11 = pd.read_csv('bible_search_engine/bible_queries_relevances/query11.csv')
query_11

,query,chapter,chapterid,relevance
0,What did God do to unloyal servants?,Psalms 107,585,5
1,What did God do to unloyal servants?,Acts 14,1032,1
2,What did God do to unloyal servants?,Luke 10,983,4
3,What did God do to unloyal servants?,Luke 19,992,4
4,What did God do to unloyal servants?,2 Peter 2,1158,5
5,What did God do to unloyal servants?,Psalms 82,560,2
6,What did God do to unloyal servants?,Isaiah 31,710,5
7,What did God do to unloyal servants?,Colossians 3,1110,1
8,What did God do to unloyal servants?,1 Peter 4,1155,3
9,What did God do to unloyal servants?,Luke 20,993,1


In [14]:
query_2 = pd.read_csv('bible_search_engine/bible_queries_relevances/query2.csv')
query_2

,query,chapter,chapterid,relevance
0,"If God is merciful and kind, why do people sti...",Matthew 5,934,4
1,"If God is merciful and kind, why do people sti...",Psalms 67,545,1
2,"If God is merciful and kind, why do people sti...",Psalms 41,519,3
3,"If God is merciful and kind, why do people sti...",1 Peter 4,1155,5
4,"If God is merciful and kind, why do people sti...",Psalms 82,560,4
5,"If God is merciful and kind, why do people sti...",Psalms 117,595,1
6,"If God is merciful and kind, why do people sti...",Romans 5,1051,5
7,"If God is merciful and kind, why do people sti...",Psalms 107,585,5
8,"If God is merciful and kind, why do people sti...",Job 36,472,5
9,"If God is merciful and kind, why do people sti...",Psalms 116,594,2


In [15]:
query_13 = pd.read_csv('bible_search_engine/bible_queries_relevances/query13.csv')
query_13

,query,chapter,chapterid,relevance
0,How do we get saved and know when we are saved?,Romans 5,1051,4
1,How do we get saved and know when we are saved?,Romans 6,1052,4
2,How do we get saved and know when we are saved?,1 Peter 1,1152,5
3,How do we get saved and know when we are saved?,Psalms 70,548,1
4,How do we get saved and know when we are saved?,Psalms 20,498,1
5,How do we get saved and know when we are saved?,2 Corinthians 1,1079,2
6,How do we get saved and know when we are saved?,Acts 3,1021,5
7,How do we get saved and know when we are saved?,Hebrews 11,1144,1
8,How do we get saved and know when we are saved?,Luke 11,984,1
9,How do we get saved and know when we are saved?,Psalms 54,532,2


In [16]:
query_4 = pd.read_csv('bible_search_engine/bible_queries_relevances/query4.csv')
query_4

,query,chapter,chapterid,relevance
0,Where does God show up in our lives if he is n...,Psalms 24,502,4
1,Where does God show up in our lives if he is n...,1 Corinthians 4,1066,2
2,Where does God show up in our lives if he is n...,Psalms 97,575,4
3,Where does God show up in our lives if he is n...,John 7,1004,5
4,Where does God show up in our lives if he is n...,2 Corinthians 6,1084,1
5,Where does God show up in our lives if he is n...,Matthew 4,933,3
6,Where does God show up in our lives if he is n...,Isaiah 38,717,5
7,Where does God show up in our lives if he is n...,2 Corinthians 4,1082,4
8,Where does God show up in our lives if he is n...,Amos 7,886,2
9,Where does God show up in our lives if he is n...,Colossians 3,1110,2


In [17]:
query_5 = pd.read_csv('bible_search_engine/bible_queries_relevances/query5.csv')
query_5

,query,chapter,chapterid,relevance
0,Who were the kings of Israel and were they fai...,1 Kings 4,295,4
1,Who were the kings of Israel and were they fai...,1 Chronicles 9,347,3
2,Who were the kings of Israel and were they fai...,2 Chronicles 12,379,5
3,Who were the kings of Israel and were they fai...,Exodus 1,51,1
4,Who were the kings of Israel and were they fai...,1 Chronicles 23,361,3
5,Who were the kings of Israel and were they fai...,2 Kings 3,316,5
6,Who were the kings of Israel and were they fai...,1 Kings 8,299,4
7,Who were the kings of Israel and were they fai...,2 Kings 15,328,5
8,Who were the kings of Israel and were they fai...,2 Chronicles 8,375,4
9,Who were the kings of Israel and were they fai...,2 Samuel 5,272,3


In [18]:
query_16 = pd.read_csv('bible_search_engine/bible_queries_relevances/query16.csv')
query_16

,query,chapter,chapterid,relevance
0,What does God say about spirituality?,Psalms 20,498,1
1,What does God say about spirituality?,Romans 12,1058,5
2,What does God say about spirituality?,1 Corinthians 2,1064,5
3,What does God say about spirituality?,1 John 4,1163,5
4,What does God say about spirituality?,1 Timothy 4,1123,5
5,What does God say about spirituality?,1 Corinthians 12,1074,5
6,What does God say about spirituality?,Matthew 4,933,4
7,What does God say about spirituality?,Acts 19,1037,4
8,What does God say about spirituality?,Psalms 107,585,2
9,What does God say about spirituality?,Luke 4,977,4


In [19]:
query_7 = pd.read_csv('bible_search_engine/bible_queries_relevances/query7.csv')
query_7

,query,chapter,chapterid,relevance
0,Why did Jesus die on the cross?,John 12,1009,5
1,Why did Jesus die on the cross?,Romans 8,1054,5
2,Why did Jesus die on the cross?,Acts 12,1030,1
3,Why did Jesus die on the cross?,Luke 24,997,5
4,Why did Jesus die on the cross?,1 Corinthians 15,1077,5
5,Why did Jesus die on the cross?,Romans 5,1051,5
6,Why did Jesus die on the cross?,Hebrews 10,1143,5
7,Why did Jesus die on the cross?,Acts 5,1023,5
8,Why did Jesus die on the cross?,John 18,1015,4
9,Why did Jesus die on the cross?,John 11,1008,3


In [20]:
query_18 = pd.read_csv('bible_search_engine/bible_queries_relevances/query18.csv')
query_18

,query,chapter,chapterid,relevance
0,How do we face discrimination and injustice?,Psalms 82,560,5
1,How do we face discrimination and injustice?,Lamentations 5,802,3
2,How do we face discrimination and injustice?,Hebrews 12,1145,5
3,How do we face discrimination and injustice?,Micah 2,895,1
4,How do we face discrimination and injustice?,1 Peter 4,1155,5
5,How do we face discrimination and injustice?,Zephaniah 2,908,1
6,How do we face discrimination and injustice?,Titus 3,1132,4
7,How do we face discrimination and injustice?,2 Corinthians 13,1091,4
8,How do we face discrimination and injustice?,Psalms 35,513,5
9,How do we face discrimination and injustice?,1 Peter 3,1154,5


In [21]:
query_19 = pd.read_csv('bible_search_engine/bible_queries_relevances/query19.csv')
query_19

,query,chapter,chapterid,relevance
0,Where do we find comfort and peace?,2 Corinthians 1,1079,5
1,Where do we find comfort and peace?,Psalms 46,524,5
2,Where do we find comfort and peace?,2 Thessalonians 3,1119,5
3,Where do we find comfort and peace?,Psalms 122,600,2
4,Where do we find comfort and peace?,Matthew 5,934,3
5,Where do we find comfort and peace?,Psalms 128,606,1
6,Where do we find comfort and peace?,Isaiah 35,714,5
7,Where do we find comfort and peace?,Psalms 67,545,1
8,Where do we find comfort and peace?,Psalms 20,498,4
9,Where do we find comfort and peace?,Psalms 83,561,1


In [22]:
query_10 = pd.read_csv('bible_search_engine/bible_queries_relevances/query10.csv')
query_10

,query,chapter,chapterid,relevance
0,Who are the enemies of God?,James 4,1150,5
1,Who are the enemies of God?,Psalms 82,560,4
2,Who are the enemies of God?,Psalms 24,502,4
3,Who are the enemies of God?,Jude 1,1167,5
4,Who are the enemies of God?,Psalms 41,519,3
5,Who are the enemies of God?,Nahum 1,901,5
6,Who are the enemies of God?,Psalms 83,561,4
7,Who are the enemies of God?,Colossians 3,1110,2
8,Who are the enemies of God?,2 Thessalonians 1,1117,3
9,Who are the enemies of God?,Psalms 97,575,2


In [23]:
test_queries = pd.concat([query_11, query_2, query_13, query_4, query_5, query_16, query_7, query_18, query_19, query_10], ignore_index=True)
test_queries.to_csv('bible_search_engine/bible_queries_relevances/test_queries_relevances.csv', index=None)
test_queries

,query,chapter,chapterid,relevance
0,What did God do to unloyal servants?,Psalms 107,585,5
1,What did God do to unloyal servants?,Acts 14,1032,1
2,What did God do to unloyal servants?,Luke 10,983,4
3,What did God do to unloyal servants?,Luke 19,992,4
4,What did God do to unloyal servants?,2 Peter 2,1158,5
...,...,...,...,...
528,Who are the enemies of God?,Romans 5,1051,4
529,Who are the enemies of God?,Matthew 4,933,5
530,Who are the enemies of God?,Luke 4,977,5
531,Who are the enemies of God?,Matthew 13,942,5
